In [15]:
from scipy.spatial import distance
from pygame import mixer
from imutils import face_utils
import imutils
import dlib
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import time

In [16]:
mixer.init()
mixer.music.load('./Assets/alertMusic/beep.mp3')

In [17]:
class Drowsiness:
    def eye_aspect_ratio(self, eye):
        A = distance.euclidean(eye[1], eye[5])
        B = distance.euclidean(eye[2], eye[4])
        C = distance.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    
    def drowsiness(self):
        thresh = 0.25
        frame_check = 20
        
        detect = dlib.get_frontal_face_detector()
        predict = dlib.shape_predictor("./Assets/face_predictor_landmarks/shape_predictor_68_face_landmarks.dat")
        
        (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
        
        face_cascade = cv2.CascadeClassifier('./Assets/Models/haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier('./Assets/Models/haarcascade_eye.xml')
        
        cap=cv2.VideoCapture(0)
        
        flag=0
        eye_flag = 0
        face_flag = 0
        
        while True:
            ret, frame=cap.read()
            frame = imutils.resize(frame, width=450)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            subjects = detect(gray, 0)
            #print(subjects)

            for subject in subjects:
                #print(subject)
                shape = predict(gray, subject)
                #print(shape)
                shape = face_utils.shape_to_np(shape)#converting to NumPy Array
                #print(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                #print(leftEye, rightEye)
                leftEAR = self.eye_aspect_ratio(leftEye)
                rightEAR = self.eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                if ear < thresh:
                    flag += 1
                    #print (flag)
                    if flag >= frame_check:
                        mixer.music.play()
                        cv2.putText(frame, "****************ALERT!****************", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        #cv2.putText(frame, "****************ALERT!****************", (10,325),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    flag = 0


            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            if len(faces)==0:
                    face_flag += 1
                    if face_flag >= 2*frame_check:
                        mixer.music.play()
                        cv2.putText(frame, "****************ALERT!****************", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        #cv2.putText(frame, "****************ALERT!****************", (10,325),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                face_flag = 0
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    roi_gray = gray[y:y + h, x:x + w]
                    roi_color = frame[y:y + h, x:x + w]
                    eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
                    for (ex, ey, ew, eh) in eyes:
                        cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

                    if len(eyes)<2 :
                        eye_flag += 1
                        if eye_flag >= 3*frame_check:
                            mixer.music.play()
                            cv2.putText(frame, "****************ALERT!****************", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                            #cv2.putText(frame, "****************ALERT!****************", (10,325),
                                #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    else:
                        eye_flag = 0


            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                cv2.destroyAllWindows()
                cap.release()
                break

In [18]:
class Face_check:
    def findEncodings(self,images):
        encodeList = []
        for img in images:
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            encode = face_recognition.face_encodings(img)[0]
            encodeList.append(encode)
        return encodeList
    
    def face_check(self):
        path = './Assets/images'
        images = []
        classNames = []
        myList = os.listdir(path)
        print(myList)
        for cl in myList:
            curImg = cv2.imread(f'{path}/{cl}')
            images.append(curImg)
            classNames.append(os.path.splitext(cl)[0])
        print(classNames)
        encodeListKnown = self.findEncodings(images)
        print('Pattern Found')
        timeout = time.time() + 10
        cap = cv2.VideoCapture(0)
        frames = 0
        while time.time() < timeout:
            success, img = cap.read()
            #img = captureScreen()
            imgS = cv2.resize(img, (0,0),None,0.25,0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
            for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
                #print(faceDis)
                matchIndex = np.argmin(faceDis)



                if faceDis[matchIndex] < 0.50:
                    name = classNames[matchIndex]
                    #print(name)

                else:
                    name = 'Anonymous'
                    #print(name)
                frames += 1
                y1, x2, y2, x1 = faceLoc
                y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            cv2.imshow('Webcam',img)
            cv2.waitKey(1)
        cap.release()
        cv2.destroyAllWindows()
        print(name)
        return name

In [19]:
class Vehicle:
        
    def start(self):
        print("------------------------------------------------------------------------------")
        print("Engine Started...")
        print("------------------------------------------------------------------------------")
    
    def stop(self):
        print("------------------------------------------------------------------------------")
        print("Engine Off...")
        print("Take Rest!")
        print("------------------------------------------------------------------------------")

In [20]:
class Driver(Vehicle, Drowsiness, Face_check):
    
    def __init__(self):
        self.ids = {}
        self.spl_key = 12345
        
    def add(self, name, Id):
        self.ids[Id] = name
    
    def check_driver(self, Id):
        if Id == self.spl_key:
            Vehicle.start(self)
            return 1
        else:
            if Id in self.ids.keys():
                name = Face_check.face_check(self)
                if self.ids[Id].lower()==name.lower(): 
                    print("Driver Name:", self.ids[Id])
                    Vehicle.start(self)
                    return 1

    def check_drowsiness(self):
        print("Checking Driver for Drowsiness...")
        Drowsiness.drowsiness(self)


In [21]:
driver = Driver()

In [22]:
driver.add("Manoj", 1)
driver.add("Jeeva", 2)
driver.add("Sanjay", 3)
driver.add("Mugesh", 4)

In [ ]:
attempt = 0
while attempt<5:
    Id = int(input("Enter Id: "))
    if driver.check_driver(Id):
        driver.check_drowsiness()
        driver.stop()
        break
    else:
        print("Invalid, ReEnter...")
    attempt += 1
else:
    print("Vehicle Locked")
